Ensayo en el codigo para incluir varias metricas en gridsearch

In [14]:
import os
import time
import json
import pandas as pd
import datetime as dt

In [5]:
current_dir = os.getcwd()

file_name = 'conf.json'
path = os.path.join(current_dir, f'{file_name}')
with open(path, 'r') as f:
    info_conf = json.load(f)
            
base_path = info_conf['base_path']
os.chdir(base_path)

In [16]:
import sklearn.metrics as metrics
from sklearn.externals import joblib
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

In [6]:
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import RidgeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [7]:
import scripts.funciones as funciones

C:\Users\pasal\Anaconda3\envs\PI2\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [8]:
d_ini = dt.datetime(2019,1,1)
d_fin = dt.datetime(2019,2,1)

In [9]:
data = funciones.read_clima_accidentes(d_ini, d_fin)
data_org = funciones.organizar_data_infoClima(data)


poblado = ['alejandria','altosdelpoblado',
            'astorga','barriocolombia',
            'castropol','elcastillo',
            'eldiamanteno2','elpoblado',
            'eltesoro','laaguacatala',
            'laflorida','lalinde',
            'laslomasno1','laslomasno2',
            'losbalsosno1','losbalsosno2',
            'losnaranjos','manila',
            'patiobonito','sanlucas',
            'santamariadelosangeles',
            'villacarlota']

data_org = data_org[data_org['BARRIO'].isin(poblado)]

data_org['poblado'] = data_org['BARRIO']
data_org= pd.get_dummies(data_org, columns=['poblado'])

X = data_org.drop(columns = ['TW','BARRIO','Accidente','summary'])
Y = data_org['Accidente']    

In [10]:
models = {
              'logistic':{
                          'mod':LogisticRegression(random_state = 42),
                          'par':{
                              'penalty': ('l1','l2'),
                              'solver': ('saga','lbfgs')

                          }
              }
          }

In [35]:
score = ['roc_auc','balanced_accuracy','f1']

In [46]:
for name in models:
    print('*'*80)
    print("Model: " + name)
    t_beg = time.time()

    pipeline = Pipeline([('scaler', StandardScaler()), (name,  models[name]['mod'])])
    parameters = {}          
    for par in models[name]['par']:
        aux = name + '__' +  par
        parameters[aux] = models[name]['par'][par]
    
    aux = GridSearchCV(pipeline, parameters, n_jobs = 1,
                      scoring = ['roc_auc','balanced_accuracy','f1'], 
                       verbose=2, cv = 2, refit = 'roc_auc')
    aux.fit(X, Y)
    models[name]['bestModel'] = aux.best_estimator_
    models[name]['mae'] = aux.best_score_

    selection_time = time.time() - t_beg
    res = pd.DataFrame(aux.cv_results_)  

    bAccuracy = res[res['params']==aux.best_params_]['mean_test_balanced_accuracy'].values[0]
    fScore = res[res['params']==aux.best_params_]['mean_test_f1'].values[0]
    models[name]['selection_time'] = selection_time

    print(f"El tiempo de seleccion fue: {selection_time:0.3f} s")
    print(f"El {score} de la familia {name} es: {models[name]['mae']:0.3f}")
    print(f"El b_accuracy de la familia {name} es: {bAccuracy:0.3f}")
    print(f"El fscore de la familia {name} es: {fScore:0.3f}")
    print('*'*80)

********************************************************************************
Model: logistic
Fitting 2 folds for each of 4 candidates, totalling 8 fits
[CV] logistic__penalty=l1, logistic__solver=saga .....................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\pasal\Anaconda3\envs\PI2\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.2s remaining:    0.0s


[CV] ...... logistic__penalty=l1, logistic__solver=saga, total=   2.3s
[CV] logistic__penalty=l1, logistic__solver=saga .....................


C:\Users\pasal\Anaconda3\envs\PI2\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\pasal\Anaconda3\envs\PI2\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)
C:\Users\pasal\Anaconda3\envs\PI2\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)


[CV] ...... logistic__penalty=l1, logistic__solver=saga, total=   2.3s
[CV] logistic__penalty=l1, logistic__solver=lbfgs ....................
[CV] ..... logistic__penalty=l1, logistic__solver=lbfgs, total=   0.0s
[CV] logistic__penalty=l1, logistic__solver=lbfgs ....................
[CV] ..... logistic__penalty=l1, logistic__solver=lbfgs, total=   0.0s
[CV] logistic__penalty=l2, logistic__solver=saga .....................


...

C:\Users\pasal\Anaconda3\envs\PI2\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


[CV] ...... logistic__penalty=l2, logistic__solver=saga, total=   1.5s
[CV] logistic__penalty=l2, logistic__solver=lbfgs ....................
[CV] ..... logistic__penalty=l2, logistic__solver=lbfgs, total=  10.1s
[CV] logistic__penalty=l2, logistic__solver=lbfgs ....................
[CV] ..... logistic__penalty=l2, logistic__solver=lbfgs, total=  11.5s


[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:   29.4s finished


El tiempo de seleccion fue: 32.720 s
El ['roc_auc', 'balanced_accuracy', 'f1'] de la familia logistic es: 0.737
El b_accuracy de la familia logistic es: 0.496
El fscore de la familia logistic es: 0.011
********************************************************************************


C:\Users\pasal\Anaconda3\envs\PI2\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [45]:
bAccuracy.values[0]

0.4961810305517556

In [41]:
print(f"El b_accuracy de la familia {name} es: {bAccuracy}")
print(f"El fscore de la familia {name} es: {fScore}")

El b_accuracy de la familia logistic es: 2    0.496181
Name: mean_test_balanced_accuracy, dtype: float64
El fscore de la familia logistic es: 2    0.011442
Name: mean_test_f1, dtype: float64


In [36]:
aux = GridSearchCV(pipeline, parameters, n_jobs = 1,refit = 'roc_auc',
                  scoring = score, verbose=2, cv = 2)
aux.fit(X, Y)

Fitting 2 folds for each of 4 candidates, totalling 8 fits
[CV] logistic__penalty=l1, logistic__solver=saga .....................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\pasal\Anaconda3\envs\PI2\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.7s remaining:    0.0s


[CV] ...... logistic__penalty=l1, logistic__solver=saga, total=   2.8s
[CV] logistic__penalty=l1, logistic__solver=saga .....................


C:\Users\pasal\Anaconda3\envs\PI2\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\pasal\Anaconda3\envs\PI2\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)


[CV] ...... logistic__penalty=l1, logistic__solver=saga, total=   3.0s
[CV] logistic__penalty=l1, logistic__solver=lbfgs ....................
[CV] ..... logistic__penalty=l1, logistic__solver=lbfgs, total=   0.1s
[CV] logistic__penalty=l1, logistic__solver=lbfgs ....................
[CV] ..... logistic__penalty=l1, logistic__solver=lbfgs, total=   0.1s
[CV] logistic__penalty=l2, logistic__solver=saga .....................


...

[CV] ...... logistic__penalty=l2, logistic__solver=saga, total=   2.3s
[CV] logistic__penalty=l2, logistic__solver=saga .....................


C:\Users\pasal\Anaconda3\envs\PI2\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


[CV] ...... logistic__penalty=l2, logistic__solver=saga, total=   2.4s
[CV] logistic__penalty=l2, logistic__solver=lbfgs ....................
[CV] ..... logistic__penalty=l2, logistic__solver=lbfgs, total=   6.7s
[CV] logistic__penalty=l2, logistic__solver=lbfgs ....................
[CV] ..... logistic__penalty=l2, logistic__solver=lbfgs, total=   8.0s


[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:   25.2s finished
C:\Users\pasal\Anaconda3\envs\PI2\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


GridSearchCV(cv=2, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('scaler',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('logistic',
                                        LogisticRegression(C=1.0,
                                                           class_weight=None,
                                                           dual=False,
                                                           fit_intercept=True,
                                                           intercept_scaling=1,
                                                           l1_ratio=None,
                                                           max_iter=100,
                                                           multi_class='auto',
        

In [37]:
res = pd.DataFrame(aux.cv_results_)
res[res['params']==aux.best_params_]['mean_test_balanced_accuracy']

2    0.496181
Name: mean_test_balanced_accuracy, dtype: float64

In [38]:
res[res['params']==aux.best_params_]['mean_test_f1']

2    0.011442
Name: mean_test_f1, dtype: float64